Face, a 1024 x 768 size image of a raccoon face, is used here to illustrate how k-means is used for vector quantization.

#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by downloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

### Version

In [1]:
import sklearn
sklearn.__version__

'0.18'

### Imports

In [2]:
print(__doc__)

import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from sklearn import cluster
from sklearn.utils.testing import SkipTest
from sklearn.utils.fixes import sp_version

Automatically created module for IPython interactive environment


In [3]:
if sp_version < (0, 12):
    raise SkipTest("Skipping because SciPy version earlier than 0.12.0 and "
                   "thus does not include the scipy.misc.face() image.")

try:
    face = sp.face(gray=True)
except AttributeError:
    # Newer versions of scipy have face in misc
    from scipy import misc
    face = misc.face(gray=True)

### Calculations

In [4]:
n_clusters = 5
np.random.seed(0)

X = face.reshape((-1, 1))  # We need an (n_sample, n_feature) array
k_means = cluster.KMeans(n_clusters=n_clusters, n_init=4)
k_means.fit(X)
values = k_means.cluster_centers_.squeeze()
labels = k_means.labels_

# create an array from labels and values
face_compressed = np.choose(labels, values)
face_compressed.shape = face.shape

vmin = face.min()
vmax = face.max()

### Plot Results

In [5]:
fig = tools.make_subplots(rows=1, cols=4)

def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = map(np.uint8, np.array(cmap(k*h)[:3])*255)
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
        
    return pl_colorscale

fig['layout'].update(width=900)

This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]  [ (1,3) x3,y3 ]  [ (1,4) x4,y4 ]



Original Face

In [6]:
orignal_face = go.Heatmap(z=face, showscale=False,
                          colorscale= matplotlib_to_plotly(plt.cm.gray, len(face)))

fig.append_trace(orignal_face, 1, 1)
fig['layout']['yaxis1'].update(autorange='reversed',
                               showticklabels=False, ticks='')
fig['layout']['xaxis1'].update(showticklabels=False, ticks='')

Compressed Face

In [7]:
compressed_face = go.Heatmap(z=face_compressed, showscale=False,
                          colorscale= matplotlib_to_plotly(plt.cm.gray, len(face)))

fig.append_trace(compressed_face, 1, 2)
fig['layout']['yaxis2'].update(autorange='reversed',
                               showticklabels=False, ticks='')
fig['layout']['xaxis2'].update(showticklabels=False, ticks='')

Equal Bins Face

In [8]:
regular_values = np.linspace(0, 256, n_clusters + 1)
regular_labels = np.searchsorted(regular_values, face) - 1
regular_values = .5 * (regular_values[1:] + regular_values[:-1])  # mean
regular_face = np.choose(regular_labels.ravel(), regular_values, mode="clip")
regular_face.shape = face.shape

regular_face = go.Heatmap(z=regular_face, showscale=False,
                          colorscale= matplotlib_to_plotly(plt.cm.gray, len(face)))

fig.append_trace(regular_face, 1, 3)
fig['layout']['yaxis3'].update(autorange='reversed',
                               showticklabels=False, ticks='')
fig['layout']['xaxis3'].update(showticklabels=False, ticks='')

Histogram

In [9]:
k = []
for i in range(0, len(X)):
    k.append(X[i][0])
    
hist = go.Histogram(x=k, showlegend=False,
                    marker=dict(color='rgb(211,211,211)'))

fig.append_trace(hist, 1, 4)

for center_1, center_2 in zip(values[:-1], values[1:]):
    axisline1 = go.Scatter(x=[.5 * (center_1 + center_2), .5 * (center_1 + center_2)], 
                           y=[0, 5000],
                           showlegend=False,
                           mode='lines',
                           line=dict(color='blue',
                                     width=2))
    
    fig.append_trace(axisline1, 1, 4)

for center_1, center_2 in zip(regular_values[:-1], regular_values[1:]):
    axisline2 = go.Scatter(x=[.5 * (center_1 + center_2), .5 * (center_1 + center_2)],
                           y=[0, 5000],
                           showlegend=False,
                           mode='lines',
                           line=dict(color='blue',
                                     width=2,
                                     dash='dash'))
    fig.append_trace(axisline2, 1, 4)
fig['layout']['yaxis4'].update(showticklabels=False, ticks='',
                               showgrid=False)
fig['layout']['xaxis4'].update(showticklabels=False, ticks='',
                               showgrid=False)

In [10]:
py.iplot(fig)

/home/diksha/.local/lib/python2.7/site-packages/plotly/plotly/plotly.py:236: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



The draw time for this plot will be slow for all clients.


/home/diksha/.local/lib/python2.7/site-packages/plotly/plotly/plotly.py:1443: UserWarning:

Estimated Draw Time Too Long



### License

Code source: 
    
            Gaël Varoquaux

License: 

            BSD 3 clause

In [2]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install git+https://github.com/plotly/publisher.git --upgrade
import publisher
publisher.publish(
    'Vector Quantization Example.ipynb', 'scikit-learn/plot-face-compress/', 'Vector Quantization Example | plotly',
    ' ',
    title = 'Vector Quantization Example | plotly',
    name = 'Vector Quantization Example',
    has_thumbnail='true', thumbnail='thumbnail/racoon.jpg', 
    language='scikit-learn', page_type='example_index',
    display_as='clustering', order=15,
    ipynb= '~Diksha_Gabha/2852')

  Cloning https://github.com/plotly/publisher.git to /tmp/pip-CcYFmQ-build
  Running setup.py install for publisher ... - error
    Complete output from command /usr/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-CcYFmQ-build/setup.py';exec(compile(getattr(tokenize, 'open', open)(__file__).read().replace('\r\n', '\n'), __file__, 'exec'))" install --record /tmp/pip-Vfe2Je-record/install-record.txt --single-version-externally-managed --compile:
    running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-2.7
    creating build/lib.linux-x86_64-2.7/publisher
    copying publisher/publisher.py -> build/lib.linux-x86_64-2.7/publisher
    copying publisher/__init__.py -> build/lib.linux-x86_64-2.7/publisher
    running install_lib
    creating /usr/local/lib/python2.7/dist-packages/publisher
    error: could not create '/usr/local/lib/python2.7/dist-packages/publisher': Permission denied
    
    -----------------------